In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# path = '/global/homes/e/ecastori/PostBorn/'
path = '../PostBornEma/'

In [ ]:
data = np.loadtxt(path+'FFT_k-1Pk.dat',skiprows=2)

#----------------------#
n         = data[:,0]
Re_c_n    = data[:,1]
Im_c_n    = data[:,2]
Re_nu_n   = data[:,3]
Im_nu_n   = data[:,4]
#----------------------#

c_n  = Re_c_n  + 1j * Im_c_n
nu_n = Re_nu_n + 1j * Im_nu_n

In [ ]:
### When reading Ema's files for the first time

# data = np.loadtxt(path+'Il_nu_t_new1.dat',skiprows=1)
# # data = np.loadtxt(path+'Il_nu_t.dat',skiprows=1)

# #----------------------#
# ell       = data[:,0]
# nn        = data[:,1]
# tt        = data[:,2]
# Re_I      = data[:,3]
# Im_I      = data[:,4]
# #----------------------#

# I_ = Re_I + 1j * Im_I

# # np.save('../PostBornEma/Il_nu_t_new1-ell-nn-tt.dat', np.stack((ell, nn, tt), axis=1))
# # np.save('../PostBornEma/Il_nu_t_new1-ReI-ImI.dat', np.stack((Re_I, Im_I), axis=1))

In [ ]:
data =  np.load(path+'Il_nu_t_new1-ell-nn-tt.npy')
ell       = data[:,0]
nn        = data[:,1]
tt        = data[:,2]
#----------------------#

data =  np.load(path+'Il_nu_t_new1-ReI-ImI.npy')
Re_I      = data[:,0]
Im_I      = data[:,1]
#----------------------#

I_ = Re_I + 1j * Im_I



In [ ]:
ell_ = np.unique(ell)
nu_n_= np.unique(nu_n) 
t_=np.unique(tt)

ell = ell.reshape(ell_.size, nu_n_.size, t_.size)
nn = nn.reshape(ell_.size, nu_n_.size, t_.size)
tt = tt.reshape(ell_.size, nu_n_.size, t_.size)
I_ = I_.reshape(ell_.size, nu_n_.size, t_.size)

cn2 = c_n.copy()
cn2[:-1] *=2
I_lcrt = np.expand_dims(I_, 2)*cn2.reshape(1, -1, 1, 1)


In [ ]:
data = np.loadtxt(path+'GG_weights.dat',skiprows=2)
t1   = data[:,0]
w1   = data[:,1]
assert(np.allclose(t_,t1))


In [ ]:
print(np.unique(tt)-t1)

In [ ]:
#cosmology
h         = 0.6770
omega_cdm = 0.11923
omega_b   = 0.02247
Omega_b   = omega_b/h**2
Omega_cdm = omega_cdm/h**2
Omega_m   = Omega_b+Omega_cdm
ns        = 0.96824
A0        = 2.10732*10**(-9)

#dictionary for class
cosmo_dict={'h': h,
'omega_b' : omega_b,
'omega_cdm': omega_cdm,
'A_s'    : A0,
'n_s'    : ns,
'k_pivot' : 0.05}
print(cosmo_dict)

# speed of light
c      = 299792458./1000. # km/s

#prefactor for Cl_kk computation from Cl_dd
prefac = 1.5*Omega_m*(100.)**2/c**2 #without h

In [ ]:
from classy import Class
cosmo = Class()
cosmo.set(cosmo_dict)
cosmo.compute()
cosmo_b               = cosmo.get_background()

class_z               = cosmo_b['z'][::-1]
class_chi             = cosmo_b['comov. dist.'][::-1]
class_D               = cosmo_b['gr.fac. D'][::-1]#/cosmo_b['gr.fac. D'][-1]

derivParams           = cosmo.get_current_derived_parameters(['z_rec'])
z_cmb                 = derivParams['z_rec']


from scipy.interpolate import interp1d
from scipy.interpolate import InterpolatedUnivariateSpline as ius
chi_z = interp1d(class_z,class_chi*h, bounds_error=False, fill_value=0)
z_chi = interp1d(class_chi*h,class_z, bounds_error=False, fill_value=0)   # Mpc/h
D_chi = interp1d(class_chi*h,class_D, bounds_error=False, fill_value=0)    # growth
D_z   = interp1d(class_z,class_D, bounds_error=False, fill_value=0)
# chi_cmb
chi_cmb = chi_z(z_cmb)



In [ ]:
print(chi_z(1100),chi_z(z_cmb))

In [ ]:
z = np.linspace(0,5,100)
plt.figure()
plt.plot(class_z,class_D,ls='-')
plt.plot(z,D_z(z),ls='--')
plt.plot(z,1/(1+z),ls='-.')
plt.xlim(0,5)
plt.show()
D_z(0)

In [ ]:
z=np.linspace(0.,10,200)
plt.figure()
plt.loglog(z,chi_z(z))
plt.show()
plt.figure()
plt.plot(z,D_z(z)/D_chi(chi_z(z)))
plt.show()
# plt.figure()
# plt.loglog(z,)
# plt.show()
# plt.figure()
# plt.loglog(z,z_chi(chi_z(z)))
# plt.show()

In [ ]:
D_chi(chi_z(0.1))

# Integrals here

In [ ]:
from scipy.integrate import simps, quadrature
from scipy.interpolate import interp1d
import time


In [ ]:
### Try with the auto-spectra without symmetrizing so that we check the second term of Eq. 3.05
##Also, no need to estimate the kernels again and again since they are independent of \ell and \nu
##This way we reduce calls to interpolate and stuff
##Well, some things depend on \nu so Use \nu as the third axes so that all the \chi kernels can be estimated beforehand

def Wkernel(xi, xmax):
    return (xmax - xi)/(xmax*xi) * (xmax > xi) * (1.+z_chi(xi))


r2d, t2d = np.meshgrid(t_,t_)
w11, w12 = np.meshgrid(w1,w1)
r2d, t2d = np.expand_dims(r2d, 2), np.expand_dims(t2d, 2)
w11, w12 = np.expand_dims(w11, 2), np.expand_dims(w12, 2)
I_ltrc = np.swapaxes(I_lcrt, 1, 3)


begin=time.time()

chi1fac0 = (Wkernel(r2d*chi_cmb, chi_cmb)*D_chi(r2d*chi_cmb))
chi1fac0 = chi1fac0 *(r2d*chi_cmb)**(1-nu_n_.reshape(1, 1, -1))

chi2fac00 = (Wkernel(t2d*r2d*chi_cmb, chi_cmb)*D_chi(r2d*t2d*chi_cmb))
chi2fac01 = (Wkernel(1/t2d*r2d*chi_cmb, chi_cmb)*D_chi(r2d*1/t2d*chi_cmb))
chi2fac01 = chi2fac01 * t2d**(nu_n_.reshape(1, 1, -1)-2)
chi2fac0 = chi2fac00 + chi2fac01

chifacs = w11*w12*chi1fac0* chi2fac0

result=np.zeros_like(ell_)
lmax = ell_.size
for ii  in range(ell_.size):        
    result[ii] = np.sum(chifacs*I_ltrc[ii])

Cl=chi_cmb * result*4./np.pi**2*prefac**2/8.

print('Time taken = ', time.time()-begin)

We have integrals of the form (for example, take the second term of $C_{22shift}^{\kappa g}$ i.e. eq. 4.2; drop the integral over $\ell$):
<br>
\begin{eqnarray}
Target &=& \int_0^{\chi_c}  d\chi W(\chi, \chi_c)\int_0^{\chi_0} d\chi' W_h(\chi') C_{l1}^{\Phi \Psi}(\chi', \chi) C_{l2}^{\delta\phi}(\chi', \chi) \\
&=&\int_0^{\chi_c} d\chi W(\chi, \chi_c)  \int_0^{\chi_0} d\chi' W_h(\chi')
\int_0^{\chi'} d\chi" W(\chi", \chi') C_{l1}^{\Psi \Psi}(\chi", \chi) 
\int_0^{\chi} d\chi''' W(\chi''', \chi) C_{l2}^{\delta \Psi}(\chi', \chi''') 
\end{eqnarray}

Now,
\begin{eqnarray}
C_l^{ab}(\chi_1, \chi_2) &=& \int \frac{dk}{k} j_l(k\chi_1)j_l(k\chi_2) [k^3 P^{ab}(k, \chi_1, \chi_2)] \\
&=& \sum_n c_n \chi_1 ^{-\nu_n} I_l(\nu_n, \frac{\chi_2}{\chi_1})
\end{eqnarray}

So, 
\begin{eqnarray}
Target &=& \int_0^{\chi_c} d\chi W(\chi, \chi_c)  \int_0^{\chi_0} d\chi' W_h(\chi') \\
&\times& \Big(\int_0^{\chi'} d\chi" W(\chi", \chi') \sum_n c^{ab}_n \chi ^{-\nu_n} I_{l1}^{ab}(\nu_n, \frac{\chi''}{\chi}) \Big) \\
&\times& \Big(\int_0^{\chi} d\chi''' W(\chi''', \chi) \sum_n c^{cd}_n \chi' ^{-\nu_n} I_{l2}^{cd}(\nu_n, \frac{\chi'''}{\chi'}) \Big)
\end{eqnarray}

These are coupled together in complicated way amongst the 4 $\chi$. 
<br>
I think the best way to estimate these are to save the array - 
$$
\int_0^{\chi} d\chi" W(\chi", \chi) \sum_n c_n \chi' ^{-\nu_n} I_l(\nu_n, \frac{\chi''}{\chi'}) 
$$
in the above above equation as a matrix $G(l, \chi, \chi')$.
<br>
This then reduces the evaluation of integrals to the sum - 
\begin{eqnarray}
Target &=& \int_0^{\chi_c} d\chi W(\chi, \chi_c)  \int_0^{\chi_0} d\chi' W_h(\chi') \times G^{ab}(l_1, \chi', \chi)\times G^{cd}(l_2, \chi, \chi')\\
\end{eqnarray}
which we can evaluate as a Gauss Qaud sum.

To save $G^{cd}(l_2, \chi, \chi')$, we have -

\begin{eqnarray}
G^{cd}(l, \chi, \chi') &=& \int_0^{\chi} d\chi" W(\chi", \chi) \sum_n c_n \chi' ^{-\nu_n} I_l(\nu_n, \frac{\chi''}{\chi'}) \\
t &=& \chi ''/\chi' \\
G^{cd}(l, \chi, \chi') &=& \int_0^{\chi/\chi'} dt W(t\chi', \chi) \sum_n c_n \chi' ^{1-\nu_n} I_l(\nu_n, t) \\
G^{cd}(l, \chi, \chi') &=& \sum_n  \int_0^1 dt \big( c_n(\chi', t\chi') (W(t\chi', \chi) + t^{-\nu-2} c_n(\chi', \chi'/t) W(\chi'/t, \chi) \big) \chi' ^{1-\nu_n} I_l(\nu_n, t) \\
\end{eqnarray}

__The problem here is that $t$ is fixed. And the obvious values of $\chi$ and $\chi'$ that we should take are $t \chi_{CMB}$. Hence I am a bit uncertain how well are we sampling this integral anymore in going from $\chi/\chi'$ to (0-1) limit. But it should work since that is what we do in the Cl_kk as well
__

In [ ]:
### Estimate as function of 'chi' and then integrate over \chi
###This is a test to check that we can estimate Cl_kk with this accurately, which kind of justifies that we are estimating
###The G matrix accurately.

def Wkernel(xi, xmax):
    return (xmax - xi)/(xmax*xi) * (xmax > xi) * (1.+z_chi(xi))


chimax = chi_cmb
chi1s = t_*chimax

chi1fac0 = (Wkernel(chi1s, chimax))#*D_chi(chi1s)) #Move this inside the sum

t2d = t_.reshape(1, -1, 1)
I_ltc = np.squeeze(I_ltrc)

begin = time.time()
result = []
for ii, chi in enumerate(chi1s):
    
    chifac = chi**(1-nu_n_.reshape(1, 1, -1)) * D_chi(chi) #Move this c_n inside the loop
    
    chi2fac00 = (Wkernel(t2d*chi, chimax)*D_chi(t2d*chi))
    chi2fac01 = (Wkernel(1/t2d*chi, chimax)*D_chi(1/t2d*chi))* t2d**(nu_n_.reshape(1,  -1)-2)
    chi2fac0 = chi2fac00 + chi2fac01
    fac = w1.reshape(1, -1, 1)*chi2fac0*chifac
    integrand = fac * I_ltc
    result.append(integrand.sum(axis = (1, 2)))

result = np.array(result)
print(result.shape)
result = (result*chi1fac0.reshape(-1, 1)*w1.reshape(-1, 1)).sum(axis=0)
Cl2=chimax * result*4./np.pi**2*prefac**2/8.

print('Time taken = ', time.time()-begin)

In [ ]:
# ### This works. This is same as above except needlessly fancy with evaluating \chi factors beforehand
# ### because I thought it will speed up things, but doesn't. oops. 

# def Wkernel(xi, xmax):
#     return (xmax - xi)/(xmax*xi) * (xmax > xi) * (1.+z_chi(xi))



# chimax = chi_cmb
# t2d = t_.reshape(1, -1, 1)
# r2d = t_.reshape(1, 1, -1)
# chi1s = t2d*chimax
# chi11d = t_*chimax

# chi1fac0 = (Wkernel(chi1s, chimax)*D_chi(chi1s))
# chi1nu = chi1s**(1-nu_n_.reshape(1, 1, -1))


# chi2fac00 = np.expand_dims(np.squeeze(Wkernel(t2d*r2d*chi_cmb, chi_cmb)*D_chi(r2d*t2d*chi_cmb)), 2)
# chi2fac01 = np.expand_dims(np.squeeze(Wkernel(1/t2d*r2d*chi_cmb, chi_cmb)*D_chi(r2d*1/t2d*chi_cmb)), 2)
# chi2fac01 = chi2fac01 * t_.reshape(-1, 1, 1)**(nu_n_.reshape(1, 1, -1)-2)
# chi2fac0 = chi2fac00 + chi2fac01


# I_ltc = np.squeeze(I_ltrc)

# begin = time.time()
# result = []
# for ii, chi in enumerate(chi11d):
       
#     integrand = w1.reshape(1, -1, 1)*chi2fac0[:, ii]*chi1nu[0, ii] * I_ltc
#     result.append(integrand.sum(axis = (1, 2)))

# result = np.array(result)
# result = (result*chi1fac0.reshape(-1, 1)*w1.reshape(-1, 1)).sum(axis=0)
# Cl3=chimax * result*4./np.pi**2*prefac**2/8.

# print('Time taken = ', time.time()-begin)

In [ ]:
plt.figure()
plt.loglog(ell_[:lmax],(ell_[:lmax]*(ell_[:lmax]+1.))**2*Cl[:lmax]/(2.*np.pi))
plt.loglog(ell_[:lmax],(ell_[:lmax]*(ell_[:lmax]+1.))**2*Cl2[:lmax]/(2.*np.pi))
# plt.loglog(ell_[:lmax],(ell_[:lmax]*(ell_[:lmax]+1.))**2*Cl3[:lmax]/(2.*np.pi))
plt.xlim(2,500)

In [ ]:
ll,clpp,clpp2 = np.loadtxt('class_ouput.out')

plt.figure()#
# plt.semilogx(ell_,Clv/np.interp(ell_,ll,clpp), label='nessa')
plt.semilogx(ell_,Cl/np.interp(ell_,ll,clpp), ls=":", lw=3, label='mcSpeedy')
plt.semilogx(ell_,Cl2/np.interp(ell_,ll,clpp), ls="--", lw=2, label='chi')
# plt.semilogx(ell_,Cl3/np.interp(ell_,ll,clpp), ls="-", lw=1, label='chi3')
plt.ylim(0.95,1.05)
# plt.ylim(0.9,2)
plt.grid()
plt.legend()

# save xifunc as described above

To save $G^{cd}(l_2, \chi, \chi')$, we have -

\begin{eqnarray}
G^{cd}(l, \chi, \chi') &=& \int_0^{\chi} d\chi" W(\chi", \chi) \sum_n c_n \chi' ^{-\nu_n} I_l(\nu_n, \frac{\chi''}{\chi'}) \\
t &=& \chi ''/\chi' \\
G^{cd}(l, \chi, \chi') &=& \int_0^{\chi/\chi'} dt W(t\chi', \chi) \sum_n c_n \chi' ^{1-\nu_n} I_l(\nu_n, t) \\
G^{cd}(l, \chi, \chi') &=& \sum_n  \int_0^1 dt \big( c_n(\chi', t\chi') (W(t\chi', \chi) + t^{-\nu-2} c_n(\chi', \chi'/t) W(\chi'/t, \chi) \big) \chi' ^{1-\nu_n} I_l(\nu_n, t) \\
\end{eqnarray}

__The problem here is that $t$ is fixed. And the obvious values of $\chi$ and $\chi'$ that we should take are $t \chi_{CMB}$. Hence I am a bit uncertain how well are we sampling this integral anymore in going from $\chi/\chi'$ to (0-1) limit. But it should work since that is what we do in the Cl_kk as well
__

In [ ]:
### Saving here

def Wkernel(xi, xmax):
    return (xmax - xi)/(xmax*xi) * (xmax > xi) * (1.+z_chi(xi))

I_ltc = np.squeeze(I_ltrc)
t2d = t_.reshape(1, -1, 1)

chis = t_*chi_cmb
chips = t_*chi_cmb

Gsave = []
for chimax in chis:

    chi1fac0 = (Wkernel(chips, chimax))#*D_chi(chi1s)) #Move this inside the sum

    begin = time.time()
    result = []
    for ii, chi in enumerate(chips):

        chifac = chi**(1-nu_n_.reshape(1, 1, -1)) * D_chi(chi) #Move this c_n inside the loop

        chi2fac00 = (Wkernel(t2d*chi, chimax)*D_chi(t2d*chi))
        chi2fac01 = (Wkernel(1/t2d*chi, chimax)*D_chi(1/t2d*chi))* t2d**(nu_n_.reshape(1,  -1)-2)
        chi2fac0 = chi2fac00 + chi2fac01
        fac = w1.reshape(1, -1, 1)*chi2fac0*chifac
        integrand = fac * I_ltc
        result.append(integrand.sum(axis = (1, 2)))

    Gsave.append(np.array(result).T)
    print(time.time()-begin)
    
Gsave = np.swapaxes(np.array(Gsave), 0, 1)

np.save('gsave', Gsave)